In [1]:
from google.colab import files

# Upload the file from your local machine
uploaded = files.upload()

Saving Hitters.csv to Hitters.csv


Ques 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Step 1: Generate a synthetic dataset
np.random.seed(42)

# Generate 1000 samples
n_samples = 1000

# Create 7 correlated features
x1 = np.random.rand(n_samples)
x2 = x1 + np.random.normal(0, 0.01, n_samples)  # Correlated with x1
x3 = x1 + np.random.normal(0, 0.01, n_samples)  # Correlated with x1
x4 = x1 + np.random.normal(0, 0.01, n_samples)  # Correlated with x1
x5 = x2 + np.random.normal(0, 0.01, n_samples)  # Correlated with x2
x6 = x3 + np.random.normal(0, 0.01, n_samples)  # Correlated with x3
x7 = x4 + np.random.normal(0, 0.01, n_samples)  # Correlated with x4

# Create a target variable with some noise
y = 3 * x1 + 2 * x2 + 1.5 * x3 + 0.5 * x4 + np.random.normal(0, 0.1, n_samples)

# Combine features into a DataFrame
X = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x4': x4,
    'x5': x5,
    'x6': x6,
    'x7': x7,
})

# Introduce some NaN values for demonstration (optional)
# X.iloc[::50, 0] = np.nan  # Uncomment to introduce NaNs for testing
# y[::50] = np.nan  # Uncomment to introduce NaNs for testing

# Check for NaN values
if X.isnull().values.any() or np.isnan(y).any():
    print("NaN values detected. Dropping rows with NaNs...")
    # Drop rows with NaN values
    X.dropna(inplace=True)
    y = y[~X.index].values  # Align target variable after dropping rows

# Step 2: Implement Ridge Regression using Gradient Descent
def ridge_regression(X, y, learning_rate, regularization_param, n_iterations=1000):
    m = len(y)
    y = y.reshape(-1, 1)
    X_b = np.c_[np.ones((m, 1)), X]  # Add bias term (intercept)

    theta = np.random.randn(X_b.shape[1], 1)  # Random initialization of theta
    cost_history = []

    for iteration in range(n_iterations):
        gradients = (2/m) * X_b.T.dot(X_b.dot(theta) - y) + 2 * regularization_param * theta
        theta -= learning_rate * gradients
        cost = (1/m) * np.sum((X_b.dot(theta) - y)**2) + regularization_param * np.sum(theta[1:]**2)
        cost_history.append(cost)

    return theta, cost_history

# Step 3: Evaluate the model with different hyperparameters
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
regularization_params = [10**-15, 10**-10, 10**-5, 10**-3, 0, 1, 10, 20]

best_r2_score = -np.inf
best_params = {}

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

for lr in learning_rates:
    for reg_param in regularization_params:
        theta, _ = ridge_regression(X_train.values, y_train, learning_rate=lr, regularization_param=reg_param)

        # Make predictions
        X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]  # Add bias term
        y_pred = X_test_b.dot(theta)

        # Check for NaN values in predictions
        if not np.isnan(y_pred).any() and not np.isnan(y_test).any():
            # Compute R² score
            r2 = r2_score(y_test, y_pred)

            # Print the parameters and the corresponding R² score
            print(f'Learning Rate: {lr}, Regularization: {reg_param}, R² Score: {r2:.4f}')

            # Check if we have a new best
            if r2 > best_r2_score:
                best_r2_score = r2
                best_params = {'learning_rate': lr, 'regularization_param': reg_param}
        else:
            print(f"NaN detected in predictions for LR: {lr}, Reg: {reg_param}")

print(f"\nBest Parameters: {best_params}, Best R² Score: {best_r2_score:.4f}")


Ques 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Step 1: Load the dataset
df = pd.read_csv('Hitters.csv')
print(df)

df.info()
# (a) Handle null values
# Create a mode imputer for categorical columns
mode_imputer = SimpleImputer(strategy='most_frequent')

# Apply mode imputer to all columns and create a new DataFrame
df = pd.DataFrame(mode_imputer.fit_transform(df), columns=df.columns)


# Convert categorical columns to numerical using one-hot encoding
print(df.Division.unique())
print(df.League.unique())
print(df.NewLeague.unique())


# using label encoding

# E = 1 ; W = 0
df['Division'].replace("E", 1, inplace = True)
df['Division'].replace("W", 0, inplace = True)

# A = 1 ; N = 0
df['League'].replace("A", 1, inplace = True)
df['League'].replace("N", 0, inplace = True)
df['NewLeague'].replace("A", 1, inplace = True)
df['NewLeague'].replace("N", 0, inplace = True)

print(df)

# Step 3: Separate input and output features
Y = df['Salary']
X = df.drop('Salary', axis=1)



# Step 4: Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 5: Train-test split (only perform it once)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

# Step 6: Linear regression using LSE method
A = X_train.T.dot(X_train)
B = np.linalg.inv(A)
C = B.dot(X_train.T)
beta = C.dot(Y_train)

# Predicting values on test set
Y_predict = X_test.dot(beta)

# Performance evaluation for Linear Regression
error = Y_test - Y_predict
sum_square_error = np.sum(np.power(error, 2))
mean_square_error = sum_square_error / len(Y_predict)
rms_error = np.sqrt(mean_square_error)
r2 = 1 - sum_square_error / np.sum((Y_test - np.mean(Y_test)) ** 2)

print(f"Linear Regression MSE: {mean_square_error}")
print(f"Linear Regression RMSE: {rms_error}")
print(f"Linear Regression R²: {r2}")

# Ridge regression
lamda = 0.5748
A_ridge = X_train.T.dot(X_train)
I_ridge = np.identity(A_ridge.shape[0])
B_ridge = np.linalg.inv(np.add(A_ridge, lamda * I_ridge))
C_ridge = B_ridge.dot(X_train.T)
beta_ridge = C_ridge.dot(Y_train)

# Predicting values on test set for Ridge
Y_predict_ridge = X_test.dot(beta_ridge)

# Performance evaluation for Ridge Regression
error_ridge = Y_test - Y_predict_ridge
sum_square_error_ridge = np.sum(np.power(error_ridge, 2))
mean_square_error_ridge = sum_square_error_ridge / len(Y_predict_ridge)
rms_error_ridge = np.sqrt(mean_square_error_ridge)
r2_ridge = 1 - sum_square_error_ridge / np.sum((Y_test - np.mean(Y_test)) ** 2)

print(f"Ridge Regression MSE: {mean_square_error_ridge}")
print(f"Ridge Regression RMSE: {rms_error_ridge}")
print(f"Ridge Regression R²: {r2_ridge}")

# Lasso regression
lasso_regression = Lasso(alpha=0.5478)
lasso_regression.fit(X_train, Y_train)
Y_predict_lasso = lasso_regression.predict(X_test)

# Performance evaluation for Lasso Regression
error_lasso = Y_test - Y_predict_lasso
sum_square_error_lasso = np.sum(np.power(error_lasso, 2))
mean_square_error_lasso = sum_square_error_lasso / len(Y_predict_lasso)
rms_error_lasso = np.sqrt(mean_square_error_lasso)
r2_lasso = 1 - sum_square_error_lasso / np.sum((Y_test - np.mean(Y_test)) ** 2)

print(f"Lasso Regression MSE: {mean_square_error_lasso}")
print(f"Lasso Regression RMSE: {rms_error_lasso}")
print(f"Lasso Regression R²: {r2_lasso}")


Ques 3

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the California housing dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Step 1: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 2: Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: RidgeCV with Cross-Validation
ridge_cv = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5)  # Cross-validate on 5 folds
ridge_cv.fit(X_train_scaled, y_train)

# Predictions and evaluation for RidgeCV
y_pred_ridge = ridge_cv.predict(X_test_scaled)
print("RidgeCV Best Alpha:", ridge_cv.alpha_)
print("RidgeCV R² Score:", r2_score(y_test, y_pred_ridge))
print("RidgeCV RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_ridge)))

# Step 4: LassoCV with Cross-Validation
lasso_cv = LassoCV(alphas=[0.1, 1.0, 10.0], cv=5)  # Cross-validate on 5 folds
lasso_cv.fit(X_train_scaled, y_train)

# Predictions and evaluation for LassoCV
y_pred_lasso = lasso_cv.predict(X_test_scaled)
print("LassoCV Best Alpha:", lasso_cv.alpha_)
print("LassoCV R² Score:", r2_score(y_test, y_pred_lasso))
print("LassoCV RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lasso)))
